In [8]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os

os.chdir(r'C:\Users\zhiha\Desktop\Temp\try')

# Constants
k = 8.987155e9          # Coulomb constant [N m² C⁻²]
dt = 0.5 * 1e-15              # Time step in seconds
num_steps = 200000       # Number of integration steps

# Masses (in kg)
m1 = 32.0 * 1.67377e-27  # Mass of ion 1
m2 = 16.0 * 1.67377e-27  # Mass of ion 2
m3 = 16.0 * 1.67377e-27  # Mass of ion 3

# Charges (in Coulombs)
Q1 = 3 * 1.602e-19      # Charge of ion 1
Q2 = 2 * 1.602e-19      # Charge of ion 2
Q3 = 2 * 1.602e-19      # Charge of ion 3

In [11]:
def calculate_acceleration(r1, r2, r3, Q1_t, Q2_t, Q3_t):
    """
    Calculate the acceleration of each ion due to Coulomb forces.
    """
    # Pre-calculate difference vectors and their norms
    diff12 = r1 - r2
    diff13 = r1 - r3
    diff21 = -diff12   # r2 - r1
    diff23 = r2 - r3
    diff31 = -diff13   # r3 - r1
    diff32 = -diff23   # r3 - r2

    # Compute norms raised to the third power
    norm12_cubed = np.linalg.norm(diff12)**3
    norm13_cubed = np.linalg.norm(diff13)**3
    norm23_cubed = np.linalg.norm(diff23)**3

    # Calculate accelerations using Coulomb force law
    a1 = k / m1 * (Q1_t * Q2_t * diff12 / norm12_cubed +
                   Q1_t * Q3_t * diff13 / norm13_cubed)
    a2 = k / m2 * (Q2_t * Q1_t * (-diff12) / norm12_cubed +
                   Q2_t * Q3_t * diff23 / norm23_cubed)
    a3 = k / m3 * (Q1_t * Q3_t * (-diff13) / norm13_cubed +
                   Q2_t * Q3_t * (-diff23) / norm23_cubed)
    return a1, a2, a3

def calculate_first_step_r(r, v, a):
    """Calculate the position at the first time step using the Verlet algorithm."""
    return r + v * dt + 0.5 * a * dt**2

def simulate_run(R, Theta):
    """
    Simulate the system for given initial geometry (R in Å and Theta in °).
    The Verlet integration loop is executed, and only the final momenta
    and OSO angle are printed and returned.
    """
    factor = 1e-10  # Conversion from Å to m

    # Initial positions (in meters)
    r1 = np.array([0.0, 0.0])
    angle_rad = math.radians(Theta * 0.5)
    r2 = np.array([R * math.sin(angle_rad), -R * math.cos(angle_rad)]) * factor
    r3 = np.array([-R * math.sin(angle_rad), -R * math.cos(angle_rad)]) * factor

    # Initial velocities (m/s)
    v1 = np.array([0.0, 0.0])
    v2 = np.array([0.0, 0.0])
    v3 = np.array([0.0, 0.0])

    # Initial acceleration and first step positions
    a1, a2, a3 = calculate_acceleration(r1, r2, r3, Q1, Q2, Q3)
    r1_new = calculate_first_step_r(r1, v1, a1)
    r2_new = calculate_first_step_r(r2, v2, a2)
    r3_new = calculate_first_step_r(r3, v3, a3)

    # Initialize positions for Verlet integration
    r1_old, r2_old, r3_old = r1, r2, r3
    r1_now, r2_now, r3_now = r1_new, r2_new, r3_new

    # Verlet integration loop
    for step in range(2, num_steps):
        a1_now, a2_now, a3_now = calculate_acceleration(r1_now, r2_now, r3_now, Q1, Q2, Q3)
        r1_new = 2 * r1_now - r1_old + a1_now * dt**2
        r2_new = 2 * r2_now - r2_old + a2_now * dt**2
        r3_new = 2 * r3_now - r3_old + a3_now * dt**2

        # Update velocities (central difference)
        v1 = 0.5 * (r1_new - r1_old) / dt
        v2 = 0.5 * (r2_new - r2_old) / dt
        v3 = 0.5 * (r3_new - r3_old) / dt

        # Prepare for next iteration
        r1_old, r2_old, r3_old = r1_now, r2_now, r3_now
        r1_now, r2_now, r3_now = r1_new, r2_new, r3_new

    # # Final momenta (in kg·m/s)
    # P1 = m1 * np.linalg.norm(v1)
    # P2 = m2 * np.linalg.norm(v2)
    # P3 = m3 * np.linalg.norm(v3)
    # # Convert momenta to atomic units (1 a.u. = 1.99285e-24 kg·m/s)
    # P1_au = P1 / (1.99285e-24)
    # P2_au = P2 / (1.99285e-24)
    # P3_au = P3 / (1.99285e-24)


    # Calculate momentum components for each ion
    p1_x_au = m1 * v1[0]/ (1.99285e-24)
    p1_y_au = m1 * v1[1]/ (1.99285e-24)
    p2_x_au = m2 * v2[0]/ (1.99285e-24)
    p2_y_au = m2 * v2[1]/ (1.99285e-24)
    p3_x_au = m3 * v3[0]/ (1.99285e-24)
    p3_y_au = m3 * v3[1]/ (1.99285e-24)



    # Calculate OSO angle (in degrees) from ion 3's final velocity.
    OSO_angle = -math.degrees(math.atan2(v3[0], v3[1]))

    # Print final result for this run
    print(f"Final results for R = {R} Å, Theta = {Theta}°:")
    #print(f"  S momentum = {P1_au:.3e} a.u., O1 momentum = {P2_au:.3e} a.u., O2 momentum = {P3_au:.3e} a.u.")
    print(f"  OSO angle = {OSO_angle:.2f}°\n")

    return {
        'R_A': R,
        'Theta_deg': Theta,
        'momentum_S_x_au': p1_x_au,
        'momentum_S_y_au': p1_y_au,
        'momentum_O1_x_au': p2_x_au,
        'momentum_O1_y_au': p2_y_au,
        'momentum_O2_x_au': p3_x_au,
        'momentum_O2_y_au': p3_y_au,
        'OSO_angle_deg': OSO_angle
    }

In [12]:
def main_simulation():
    """
    Run the simulation for a range of R and Theta values, collect the final results,
    and save them to a CSV file.
    """
    results = []
    for R in np.arange(1.4, 3.2, 0.3):
        for Theta in np.arange(90, 140, 5):
            result = simulate_run(R, Theta)
            results.append(result)
    
    # Create DataFrame and save results to CSV
    df = pd.DataFrame(results)
    df.to_csv("final_simulation_results.csv", index=False)
    print("Final simulation results saved to final_simulation_results.csv")
    return df

if __name__ == "__main__":
    df = main_simulation()

Final results for R = 1.4 Å, Theta = 90°:
  OSO angle = 125.40°

Final results for R = 1.4 Å, Theta = 95°:
  OSO angle = 124.52°

Final results for R = 1.4 Å, Theta = 100°:
  OSO angle = 123.49°

Final results for R = 1.4 Å, Theta = 105°:
  OSO angle = 122.31°

Final results for R = 1.4 Å, Theta = 110°:
  OSO angle = 120.99°

Final results for R = 1.4 Å, Theta = 115°:
  OSO angle = 119.54°

Final results for R = 1.4 Å, Theta = 120°:
  OSO angle = 117.96°

Final results for R = 1.4 Å, Theta = 125°:
  OSO angle = 116.25°

Final results for R = 1.4 Å, Theta = 130°:
  OSO angle = 114.41°

Final results for R = 1.4 Å, Theta = 135°:
  OSO angle = 112.46°

Final results for R = 1.7 Å, Theta = 90°:
  OSO angle = 125.40°

Final results for R = 1.7 Å, Theta = 95°:
  OSO angle = 124.52°

Final results for R = 1.7 Å, Theta = 100°:
  OSO angle = 123.49°

Final results for R = 1.7 Å, Theta = 105°:
  OSO angle = 122.31°

Final results for R = 1.7 Å, Theta = 110°:
  OSO angle = 120.99°

Final results 

KeyboardInterrupt: 